In [2]:

from pymongo import MongoClient
from langchain.agents import tool
from typing import Sequence, Annotated, TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import ToolMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import END, StateGraph
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_community.document_transformers.openai_functions import create_metadata_tagger
from langchain_core.documents.base import Document
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel
from setting import settings
import os
from langchain_core.utils.utils import convert_to_secret_str


In [11]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", api_key=settings.GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", google_api_key=convert_to_secret_str(settings.GOOGLE_API_KEY))

dbname = "africonomy"
collection_name = "africa_outlook"
client = MongoClient(settings.MONGO_URI)
db = client[dbname]
collection = db[collection_name]

In [8]:
class AgentState(TypedDict):
    messages:Annotated[Sequence[BaseMessage], add_messages]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [12]:
file_dir = os.path.join(os.path.expanduser("~"), "Desktop")
file_name = "africa_econ.pdf"
file_path = os.path.join(file_dir, file_name)

if not os.path.isfile(file_path):
    raise FileNotFoundError("The file is not found in provided directory")

try:
    pages = PyPDFLoader(file_path).load()
except Exception as e:
    print(str(e))


In [15]:
docs = text_splitter.split_documents(pages)

vector_store = MongoDBAtlasVectorSearch.from_documents(
    docs,
    embeddings,
    collection=collection
)

In [16]:
vector_index_model = SearchIndexModel(
    name="afronomy_index",
    type="vectorSearch",
    definition={
        "fields":[{
            "type":"vector",
            "path":"embeddings",
            "numDimensions":3072,
            "similarity":"cosine",
            "quantization":"scalar"
        },
        {
            "type": "filter",
            "path": "description"
        }
        ]
    }
)

result = collection.create_search_index(model=vector_index_model)
print(result)

afronomy_index


In [17]:
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=settings.MONGO_URI,
    namespace=f"{dbname}.{collection_name}",
    embedding=embeddings,
    index_name="afronomy_index"
)

In [18]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k":80,
        "score_threshold":0.04
    }
)

In [21]:
@tool
def vector_retrival(query:str)->str:
    """Uses the query to perform vector search on the specified mongodb database"""
    results = retriever.invoke(query)

    if not results:
        return "Vector search returned nothing"
    
    return "\n".join(results)

tools = [vector_retrival]

tool_dict = {tool.name:tool for tool in tools}

sys_prompt = """You are an intelligent research assistant. You will assist the user on the research paper named African Economic Outlook
- If the user asks questions not in context of the research paper, inform the user that you are only to research based on the paper
- If user needs to provide more details for you to the search tool, inform the user as such.
"""

In [22]:
def agent_call(state:AgentState) -> AgentState:
    """Invokes the llm with state messages"""

    messages = [SystemMessage(content=sys_prompt)] + list(state['messages'])

    response = llm.invoke(messages)

    print("\n AI: ", response.content)

    return {"messages": [response]}

In [23]:
def should_continue(state:AgentState)->bool:
    """Returns a bool value. True if last message in state has tool_calls attribute 
    and the length of the tool_calls list is greater than 0 else returns false.
    """

    last_msg = state['messages'][-1]

    return hasattr(last_msg, "tool_calls") and len(last_msg.tool_calls) > 0

In [24]:
def tool_call(state:AgentState)->AgentState:
    """Invokes the tool and passes the result to state. Also checks if tool exists and llm is not hallucinating."""

    last_msg = state['messages'][-1]

    tool_calls = last_msg.tool_calls

    results = []

    for tool in tool_calls:
        if not tool['name'] in tool_dict:
            result = "Tool doesn't exist in tools list"

        else:
            result = tool_dict[tool['name']].invoke(tool['args'].get('query', ''))

        results.append(ToolMessage(tool_call_id=tool['id'], name=tool['name'], content=str(result)))

    return {"messages": results}

In [25]:
graph = StateGraph(AgentState)

graph.add_node('tools', tool_call)
graph.add_node('agent', agent_call)
graph.add_edge('tools', 'agent')
graph.set_entry_point('agent')

graph.add_conditional_edges(
    'agent',
    should_continue,
    {
        True:'tools',
        False:END
    }
)

agent = graph.compile()

In [26]:
user_input = input("\n Enter your questions about the African Economic Outlook Paper: ")

while not user_input.lower() in ['exit', 'end', 'quit']:
    print("\n Human: ", user_input)
    agent.invoke({'messages': [HumanMessage(content=user_input)]})

    user_input = input("\n Enter your questions about the African Economic Outlook Paper: ")



 Human:  What is the research about

 AI:  This research paper is the African Economic Outlook.

 Human:  What is the highlight of the research

 AI:  The African Economic Outlook research paper highlights that Africa's economy is projected to grow at 4% in 2023 and 4.3% in 2024.

 Human:  According to the paper, what are some of the challenges facing African economy

 AI:  According to the paper, some of the challenges facing the African economy include:

*   Macroeconomic stability
*   Structural transformation
*   Private sector development
*   Human capital
*   Governance
*   Climate change
*   Debt sustainability

 Human:  expansiate on the chanllenges

 AI:  Please specify which challenges you're interested in. Are you referring to specific sectors, countries, or types of challenges (e.g., economic, social, environmental)? Providing more context will help me give you a more detailed explanation.

 Human:  debt sustainability

 AI:  To provide you with information on debt sustain